In [1]:
import csv
import glob
import os
import pandas as pd
import re
from  dateutil.parser import parse

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [2]:
def load_data(file):
    """
    Loads a CSV file into a dataframe and drops NaN columns
    input: Path to a csv file
    output: Dataframe
    """
    # Find the header row
    num_rows = find_header_row(file)
    df = pd.read_csv(file, skiprows=num_rows)
    # Drop columns where all values are NaN
    df = df.dropna(axis='columns', how='all')
    return df


def find_header_row(file):
    """
    Read each line in a CSV file and return a row number for the header
    input: Path to a csv file
    output: integer (the row number)
    """
    with open(file,'r') as infile:
        reader = csv.reader(infile, delimiter=',', quotechar='"')
        row_num = 0
        for row in reader:
            if (row[0] == 'ID') | (row[0] == 'TR ID#') | (row[0] == 'Legal First Name'):
                break
            else:
                row_num += 1
                next
    return(row_num)


def write_csv(file, df):
    """
    Writes a CSV file to the "cleaned" directory
    input: a cleaned dataframe (after load_data())
    output: csv file
    """
    path_cleaned = 'cleaned/'
    os.makedirs(path_cleaned, exist_ok=True)
    path_cleaned_file = path_cleaned+file
    df.to_csv(path_cleaned_file, index=None)
    
    
def parse_op_name(filename):
    """
    Parses an operation name from a mobtool's file name
    """
    filename_parts = re.split(', | - ', filename)
    for part in filename_parts:
        part = part.upper()
        # Strip superfluous text, punctuation, and characters
        if "op".upper() in part:
            op_name = part.replace("PERSONNEL PLANNING TOOL", "").replace("MOBILIZATION TOOL","")
            op_name = op_name.replace("_", "").replace("'","")
            if "operation".upper() not in op_name:
                op_name= op_name.replace("op".upper(), "Operation".upper())            
    return op_name


def standardize_dates(df):
    mapper={}
    for col in df.columns:
        try:
            dt = parse(str(col))
            new_date = dt.strftime('%Y-%m-%d')
            mapper[col] = new_date 
        except:
            next
    df.rename(mapper, axis=1, inplace=True)
    return df


def get_start_end_dates(series):
    """
    Returns dates a volunteer first and last deployed. Also returns total days deployed.
    This does not take into account that volunteers can deploy multiple times on an op and 
    does not count travel days.
    
    input: a Pandas series (dataframe row) 
    output: tuple, first date, last date of deployment, and total days deployed
    
    This requires the dates to be standardized and numercial
    """
    df = series.to_frame().transpose()
    
    dropcols=[]
    for col in df.columns:
        try: 
            int(col[0])        
        except:
            dropcols.append(col)
    df.drop(columns=dropcols, inplace=True)
    df.dropna(axis=1, inplace=True)
    
    dates=[]
    for col in df.columns:
        dt = parse(str(col))
        dates.append(dt.strftime('%Y-%m-%d'))
    if len(dates) > 0:
        start = min(dates)
        end = max(dates)
        total_days_deployed = len(df.columns)
        return start, end, total_days_deployed
    else:
        return None, None, None
    
    
def merge_dataframes(dfs):
    final_columns = [
        'TR ID#', 'OP_NAME', 'START_DATE', 'END_DATE', 'TOTAL_DAYS',
        'TOTAL_HOURS', 'IMPACT_DOLLARS', 'First Name', 'Last Name',
        'Legal First Name', 'Email', 'Phone #', 'Position', 'Contact Number (ex:143.143.1234)']
    final_df = pd.concat(dfs, axis=0, join="outer", sort=True)
    final_df = final_df[final_columns]
    
    # Drop rows where these columns are all Null values 
    # This cleans up GSheets that have tons of extra empty rows
    drop_columns = ['TR ID#','First Name', 'Last Name', 'Legal First Name', 'Email', 
        'Phone #', 'Position', 'Contact Number (ex:143.143.1234)']
    
    final_df.dropna(subset=drop_columns, how='all', inplace=True)
    return final_df

In [3]:
def main(input_dir):
    if "/" not in input_dir:
        input_dir = input_dir + "/"
    input_files = [f.strip("input/") for f in glob.glob(input_dir+'/*.csv')]
    
    dfs=[]
    for file in input_files:
        # Load and clean
        df = load_data(input_dir+file)
           
        # Parse Op name and add to the dataframe
        op_name = parse_op_name(file)
        df['OP_NAME'] = op_name
        print(op_name)
        
        # Standardize the format of all the dates
        df = standardize_dates(df)

        # Get the dates
        for row in range(len(df)):
            series = df.iloc[row]
            start, end, total_days_deployed = get_start_end_dates(series)
            df.at[row,'START_DATE'] = str(start)
            df.at[row,'END_DATE'] = str(end)
            df.at[row,'TOTAL_DAYS'] = total_days_deployed
            
        # Total hours based on standard 10 hour day
        df['TOTAL_HOURS'] = df['TOTAL_DAYS'] * 10 # Standarad 10 hour day
        
        # Total Impact dollars
        df['IMPACT_DOLLARS'] = df['TOTAL_HOURS'] * 28.15
        
        # Add dataframes to list
        dfs.append(df)        
        
        # Write to file
        write_csv(file, df)
    
    # Concat all dataframes
    final_df = merge_dataframes(dfs)
    
    # Write full dataset to file
    write_csv('final_df.csv', final_df)
        
    print("Done.")
    return dfs, final_df

dfs, final_df = main('input/')

OPERATION SKIPPING CHRISTMAS
OPERATION CRAZY TRAIN
OPERATION COAL MINERS DAUGHTER
OPERATION DOUBLE TROUBLE
OPERATION OLD PUT
OPERATION TWISTED TRUNK 
OPERATION HUCKLEBERRY HUSTLE
OPERATION PALMETTO PUNCH 
OPERATION RIGHT STUFF
OPERATION BARBED WIRE
OPERATION OLD ANCHOR
OPERATION BIG DIG
OPERATION BREDO RISING 
OPERATION SLEEPING BEAR
Done.


Total Hours (based on standard 10 Hour Day)
Total Impact Dollars (Based on total hours X $28.15 per hour)

In [4]:
df = dfs[1]
df.head()

df.head()

,Legal First Name,Last Name,Email,Contact Number (ex:143.143.1234),Skill / Position (One Word),Smartwaiver Signed?,2018-06-26,2018-06-27,2018-06-28,2018-06-29,2018-06-30,2018-07-01,2018-07-02,2018-07-03,2018-07-04,2018-07-05,2018-07-06,2018-07-07,2018-07-08,2018-07-09,2018-07-10,2018-07-11,2018-07-12,2018-07-13,2018-07-14,2018-07-15,2018-07-16,2018-07-17,2018-07-18,2018-07-21,2018-07-22,OP_NAME,START_DATE,END_DATE,TOTAL_DAYS,TOTAL_HOURS,IMPACT_DOLLARS
0,William (TJ),Porter,porter@teamrubiconusa.org,843-633-1510,IC,YES,NaN,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN,NaN,NaN,OPERATION CRAZY TRAIN,2018-06-27,2018-07-17,21.0,210.0,5911.5
1,Don,Iacobazzi,d.iacobazzi@yahoo.com,630-202-6343,OSC,YES,NaN,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OPERATION CRAZY TRAIN,2018-06-27,2018-07-12,16.0,160.0,4504.0
2,Mikey,Hernandez,hernandez@teamrubiconusa.org,224-388-7766,Recon,YES,NaN,X,X,NaN,NaN,NaN,NaN,X,X,X,X,X,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OPERATION CRAZY TRAIN,2018-06-27,2018-07-12,8.0,80.0,2252.0
3,Jack,Ginther,jack.ginther@teamrubiconusa.org,740-396-9800,Recon,YES,X,X,X,X,NaN,NaN,NaN,X,NaN,NaN,X,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OPERATION CRAZY TRAIN,2018-06-26,2018-07-10,7.0,70.0,1970.5
4,Maggi,Thomas,thomas@teamrubiconusa.org,815-883-0752,PSC,YES,X,X,X,X,X,X,X,X,X,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OPERATION CRAZY TRAIN,2018-06-26,2018-07-05,10.0,100.0,2815.0
